In [2]:
#dependencies
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
import gym


Using TensorFlow backend.


## Agent class

In [10]:
class Agent:
    def __init__(self, envName = ('CartPole-v0'), gamma = None, epsilon = None):
            self.envName = envName
            self.env = gym.make(envName)
            self.memory = deque(maxlen=10000)
            self.gamma = gamma
            self.epsilon = epsilon
            self.e_decay = .99
            self.e_min = 0.01  # minimal exploraton rate
            self.learning_rate = 0.01
            self.model = self._build_model()

    def _build_model(self):
            model = Sequential()
            model.add(Dense(20, input_dim=self.env.observation_space.shape[0], activation='tanh'))
            model.add(Dense(20, activation='tanh', init='uniform'))
            model.add(Dense(self.env.action_space.n, activation='linear'))
            model.compile(loss='mse', optimizer=RMSprop(lr=self.learning_rate))
            return model

    def remember(self, state, action, reward, next_state, done):
            self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
            if np.random.rand() <= self.epsilon:
                return random.randrange(self.env.action_space.n)
            act_values = self.model.predict(state)
            return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        batch_size = min(batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)
        X = np.zeros((batch_size, self.env.observation_space.shape[0]))
        Y = np.zeros((batch_size, self.env.action_space.n))
        for i in range(batch_size):
            state, action, reward, next_state, done = minibatch[i]
            target = self.model.predict(state)[0]
            print "i", i, " predict", self.model.predict(state)
            if done:
                target[action] = reward
            else:
                target[action] = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            X[i], Y[i] = state, target
        self.model.fit(X, Y, batch_size=batch_size, nb_epoch=1, verbose=0)
        if self.epsilon > self.e_min:
            self.epsilon *= self.e_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)



# Model

In [7]:
agent = Agent('CartPole-v0',.99,1.0)
# Iterate the game
for e in range(1000):
    state = agent.env.reset()
    state_size = agent.env.observation_space.shape[0]
    state = np.reshape(state, [1,state_size])
    resultReward = 0
    while True:
        agent.env.render()
        action = agent.act(state)
        next_state, reward, done, _ = agent.env.step(action)
        # reward = reward if not done else -1
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        resultReward += reward
        if done:
            print "episode:", e, "score:", resultReward #, 'ep', e
            break
    agent.replay(32)
    if e % 10 == 0:
        agent.save("cartpole3.kek")

[2017-10-10 14:20:49,909] Making new env: CartPole-v0
/Users/zelenov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="uniform", activation="tanh")`


episode: 0 score: 11.0
episode: 1 score: 12.0
episode: 2 score: 12.0
episode: 3 score: 22.0
episode: 4 score: 18.0
episode: 5 score: 48.0
episode: 6 score: 20.0
episode: 7 score: 23.0
episode: 8 score: 28.0
episode: 9 score: 10.0
episode: 10 score: 22.0
episode: 11 score: 31.0
episode: 12 score: 16.0
episode: 13 score: 33.0
episode: 14 score: 21.0
episode: 15 score: 20.0
episode: 16 score: 13.0
episode: 17 score: 15.0
episode: 18 score: 13.0
episode: 19 score: 39.0
episode: 20 score: 19.0
episode: 21 score: 10.0
episode: 22 score: 60.0
episode: 23 score: 18.0
episode: 24 score: 32.0
episode: 25 score: 13.0
episode: 26 score: 18.0
episode: 27 score: 106.0
episode: 28 score: 10.0
episode: 29 score: 31.0
episode: 30 score: 25.0
episode: 31 score: 14.0
episode: 32 score: 30.0
episode: 33 score: 33.0
episode: 34 score: 24.0
episode: 35 score: 22.0
episode: 36 score: 12.0
episode: 37 score: 14.0
episode: 38 score: 21.0
episode: 39 score: 15.0
episode: 40 score: 28.0
episode: 41 score: 12.0
e

# Atother model

In [11]:
agent = Agent('CartPole-v0',.99,1.0)
# Iterate the game
for e in range(1000):
    state = agent.env.reset()
    state_size = agent.env.observation_space.shape[0]
    state = np.reshape(state, [1,state_size])
    resultReward = 0
    while True:
        agent.env.render()
        action = agent.act(state)
        next_state, reward, done, _ = agent.env.step(action)
        # reward = reward if not done else -1
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        resultReward += reward
        if done:
            print "episode:", e, "score:", resultReward #, 'ep', e
            break
    agent.replay(32)
    if e % 10 == 0:
        agent.save("cartpole3.kek")

[2017-10-10 15:22:12,067] Making new env: CartPole-v0
/Users/zelenov/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="uniform", activation="tanh")`


episode: 0 score: 35.0
i 0  predict [[ 0.03806543 -0.0062082 ]]
i 1  predict [[ 0.00162874  0.00209704]]
i 2  predict [[ 0.01136567 -0.00526931]]
i 3  predict [[ 0.01687608  0.00937848]]
i 4  predict [[ 0.04560965  0.00509724]]
i 5  predict [[-0.01531599 -0.00485749]]
i 6  predict [[ 0.0191083   0.00744007]]
i 7  predict [[ 0.02324399  0.003902  ]]
i 8  predict [[ 0.02580624  0.00229413]]
i 9  predict [[ 0.03543679  0.01263483]]
i 10  predict [[ 0.02874146  0.00049046]]
i 11  predict [[ 0.00170076  0.00225385]]
i 12  predict [[ 0.05525975 -0.00205877]]
i 13  predict [[-0.01458914 -0.00471571]]
i 14  predict [[ 0.00066641  0.00288099]]
i 15  predict [[ 0.03407329  0.01404628]]
i 16  predict [[-0.01596248 -0.00469906]]
i 17  predict [[ 0.01725009  0.00894496]]
i 18  predict [[ 0.00142504  0.00200998]]
i 19  predict [[ 0.03740827  0.01087205]]
i 20  predict [[ 0.05013659  0.0014759 ]]
i 21  predict [[ 0.00854736 -0.00356542]]
i 22  predict [[-0.01534617 -0.0042261 ]]
i 23  predict [[ 0.06

KeyboardInterrupt: 